In [2]:
#Summarize with context
import json

# Load the dictionary from JSON file
with open("simple_book_summary.json", 'r', encoding='utf-8') as fp:
    book_dict = json.load(fp)



In [66]:

from openai import OpenAI
from tqdm import tqdm

client = OpenAI(api_key=api)
SYSTEM_INSTRUCTION = "You are a helpful assistant that summarizes paragraphs from books. Return markdown formatting without any tags around it."
# Function to summarize text using ChatGPT
def summarize_text(text=None, messages=None):

    if text is not None and messages is not None:
        raise Exception()

    if text is None and messages is None:
        raise Exception()

    if messages is None:
        messages=[
            {"role": "system", "content": SYSTEM_INSTRUCTION },
            {"role": "user", "content": f"Summarize the following text: {text}"}
        ]
    
    response = client.chat.completions.create(
        model="gpt-4o", 
        messages = messages,
    )
    summary = response.choices[0].message.content.strip()
    return summary, messages

In [20]:
# Enrich Chapter Summary
def deep_copy_dict(dictionary):
    """
    Create a deep copy of a dictionary, handling nested dictionaries, lists, and other objects.
    
    Args:
        dictionary (dict): The input dictionary to copy
        
    Returns:
        dict: A deep copy of the input dictionary
    """
    # Base case - if input is not a dictionary
    if not isinstance(dictionary, dict):
        # If it's a list, deep copy each element
        if isinstance(dictionary, list):
            return [deep_copy_dict(item) for item in dictionary]
        # If it's any other type, return as is (assuming immutable)
        return dictionary
    
    # Create new dictionary for the copy
    copy = {}
    
    # Iterate through key-value pairs
    for key, value in dictionary.items():
        # Recursively copy nested dictionaries/lists
        copy[key] = deep_copy_dict(value)
        
    return copy

book_dict_sections = deep_copy_dict(book_dict)

In [22]:
book_summary = book_dict['summary']['text']



for c_title, chapter in book_dict['content'].items():
    print(c_title)

    internal_text = ""
    for section in chapter['content']:
        section_title = section['title']
        summary_text = section['summary']['text']
        
        internal_text += f"\n##{section_title}" + "\n" + summary_text

    instruction = "Summarize the following text composed of summaries of each subsection of the chapter."
    instruction += f"Take into account the broader context of the book while still focusing on the chapter, this context comes from its summary:\n {book_summary}"
    instruction += f"\nThe chapter is '{c_title}' and its contents: {internal_text}"            
    messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes paragraphs from books. Use paragraphs to break logically and to enable better structure. Return markdown formatting without any tags around it."},
            {"role": "user", "content": instruction}
    ]

    chapter_summary_text, messages = summarize_text(internal_text, messages=messages)
    
    strategy = "Take the collected summaries for subsections, the previous book summary and then summarize using: " + str(messages)

    chapter = {"summary": 
                   {"strategy": strategy,
                    "text": chapter_summary_text},
                "content": chapter['content']}

    book_dict['content'][c_title] = chapter

chapter['summary'][

Introduction
Konrad Adenauer: The Strategy of Humility
Charles de Gaulle: The Strategy of Will
Richard Nixon: The Strategy of Equilibrium
Anwar Sadat: The Strategy of Transcendence
Lee Kuan Yew: The Strategy of Excellence
Margaret Thatcher: The Strategy of Conviction
Conclusion: The Evolution of Leadership


'The chapter "Conclusion: The Evolution of Leadership" in Henry Kissinger\'s book "Leadership" revisits Konrad Adenauer\'s 1967 questioning of contemporary leaders\' capacity for long-range policies and true leadership. Assessing six major 20th-century leaders, the text considers if similar figures with essential qualities can emerge today to address global challenges. Historically, leaders often arose in times of crisis rather than tranquility, which cultivates societal complacency. Philosophers like Machiavelli and Weber highlight the necessity of resilience, strategic insight, and societal focus in leadership, suggesting that genuine leadership emerges from the interplay of fixed circumstances and individual agency.\n\nKey insights are drawn from historical examples, showing that capable leaders like Sadat, Lee, and Thatcher emerged during crises, challenging peaceful periods that often lower leadership standards. The chapter argues that leadership\'s role involves maintaining faith

In [23]:
print(chapter['summary']['text'])

The chapter "Conclusion: The Evolution of Leadership" in Henry Kissinger's book "Leadership" revisits Konrad Adenauer's 1967 questioning of contemporary leaders' capacity for long-range policies and true leadership. Assessing six major 20th-century leaders, the text considers if similar figures with essential qualities can emerge today to address global challenges. Historically, leaders often arose in times of crisis rather than tranquility, which cultivates societal complacency. Philosophers like Machiavelli and Weber highlight the necessity of resilience, strategic insight, and societal focus in leadership, suggesting that genuine leadership emerges from the interplay of fixed circumstances and individual agency.

Key insights are drawn from historical examples, showing that capable leaders like Sadat, Lee, and Thatcher emerged during crises, challenging peaceful periods that often lower leadership standards. The chapter argues that leadership's role involves maintaining faith in the

In [42]:
book_summary = book_dict['summary']['text']

for c_title, chapter in book_dict_sections['content'].items():
    chapter_summary = chapter['summary']['text']
    print(c_title)
    section_summary_text = ""
    for i, section in enumerate(chapter['content']):     
        internal_text = "\n".join(section['original_text'][0])
        title = section['title']
        print(f"\t{title}")
        
        instruction = "Summarize the following text composed of paragraphs of a section of a chapter."
        instruction += f"Take into account the broader context of the book while still focusing on the section mainly." 
        instruction += f"\nThis context comes from its summary:\n {book_summary}"
        instruction += f"\nAnd from the chapter summary:\n {chapter_summary}"
        if section_summary_text:
            instruction += f"\n Also take into account the previous section in order for it to flow properly: {section_summary_text}"
        else:
            instruction += "This is the first section of this chapter"
        instruction += "Take into account that each the resulting summary will be used preceded by it's title so avoid mentioning it."
        instruction += "It will also be grouped within it's chapter surrounded by the neighbouring section summaries, this should be considered to make sure the text flows properly."
        instruction += f"\nThe section to summarize is '{title}' and its contents: {internal_text}"            

        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes paragraphs from books. Use paragraphs to break logically and to enable better structure. Return markdown formatting without any tags around it."},
            {"role": "user", "content": instruction}
        ]
    
        section_summary_text, messages = summarize_text(internal_text, messages=messages)
        
        strategy = "Take the collected summaries for subsections, the previous book summary and then summarize using: " + str(messages)
    
        book_dict_sections['content'][c_title]['content'][i]['summary'] = {"strategy": strategy, "text": section_summary_text}


Introduction
	The Axes of Leadership
	The Nature of Leadership Decisions
	Six Leaders in Their Context
	Epitomes of Leadership: The Statesman and the Prophet
	The Individual in History
Konrad Adenauer: The Strategy of Humility
	Konrad Adenauer: The Strategy of Humility
	The Necessity of Renewal
	From Early Life to Internal Exile
	The Road to Leadership
	The Restoration of Civil Order and the Inauguration of the Chancellor
	The Path to a New National Identity
	The Soviet Challenge and Rearmament
	The Inextricable Past: Reparations to the Jewish People
	Two Crises: Suez and Berlin
	Three Conversations with Adenauer
	German Unification: The Tormenting Wait
	Final Conversations
	The Adenauer Tradition
Charles de Gaulle: The Strategy of Will
	Charles de Gaulle: The Strategy of Will
	Close Encounters
	The Beginning of the Journey
	The Sources and Aims of de Gaulle’s Conduct
	De Gaulle in the History of France
	De Gaulle and the Second World War
	North African contest
	Achieving Political Pow

In [ ]:
book_summary = book_dict_sections['summary']['text']

for c_title, chapter in book_dict_sections['content'].items():
    print(c_title)

    internal_text = ""
    for section in chapter['content']:
        section_title = section['title']
        
        internal_text += f"##{section_title}" + "\n" + section['summary']['text']

    instruction = "Summarize this chapter from the summaries from within it."
    instruction += f"Take into account the broader context of the book while still focusing on the chapter. The book summary:\n {book_summary}"
    instruction += f"\nThe chapter is called '{c_title}' and its contents:\n {internal_text}"  
    instruction += f"\nYour summary should not mention the section names and should be a cohesive flowing text."

    messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes paragraphs from books. Use paragraphs to break logically and to enable better structure. Return markdown formatting without any tags around it."},
            {"role": "user", "content": instruction}
    ]

    chapter_summary_text, messages = summarize_text(messages=messages)

    #print(f"\n{chapter_summary_text}\n")
    
    strategy = "Take the collected summaries for subsections, the previous book summary and then summarize using: " + str(messages)

    chapter = {"summary": 
                   {"strategy": strategy,
                    "text": chapter_summary_text},
                "content": chapter['content']}

    book_dict_sections['content'][c_title] = chapter


In [111]:
book_title = book_dict_sections['title']

concatenated = ""
for title, chapter in book_dict_sections['content'].items():
    print(title)
    concatenated += f"## Chapter '{title}'" + "\n"
    concatenated += f"{chapter['summary']['text']}"
    concatenated += "\n\n"

messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes paragraphs from books. Use paragraphs to break logically and to enable better structure. Return markdown formatting without any tags around it."},
            {"role": "user", "content": f"Create a concluding summary (in the number of appropriate paragraphs) of the book {book_title} from the summaries of its chapters: {concatenated}"}
    ]

book_summary_text, messages = summarize_text(messages=messages)
    
strategy = "Take the collected summaries for chapters and then summarize using: " + str(messages)

book_dict_sections['summary'] = {"strategy": strategy, "text": book_summary_text}


Introduction
Konrad Adenauer: The Strategy of Humility
Charles de Gaulle: The Strategy of Will
Richard Nixon: The Strategy of Equilibrium
Anwar Sadat: The Strategy of Transcendence
Lee Kuan Yew: The Strategy of Excellence
Margaret Thatcher: The Strategy of Conviction
Conclusion: The Evolution of Leadership


In [112]:
print(book_dict_sections['summary']['text'])

In "Leadership," Henry Kissinger delves into the essence and evolution of leadership through profiles of pivotal 20th-century figures who navigated their nations through profound changes. The book underscores the indispensable role of leadership in guiding societies from their historical contexts toward uncertain futures. Leaders like Konrad Adenauer, Charles de Gaulle, Richard Nixon, Anwar Sadat, Lee Kuan Yew, and Margaret Thatcher exemplify the various strategies—humility, will, equilibrium, transcendence, excellence, and conviction—employed to address unique challenges and steer national destinies.

The narratives of these leaders illustrate common elements: the balance of past lessons with future aspirations, the ability to adapt history to current needs, and the personal character attributes necessary for enduring crises. Each leader faced distinct circumstances, yet demonstrated strategic foresight, pragmatic vision, and extraordinary resolve to foster change. From Adenauer's eff

In [143]:
import re
import markdown
from bs4 import BeautifulSoup

def text_to_paragraphs(text):
    """Convert text with newlines to HTML paragraphs, handling both plain text and markdown.
    
    Args:
        text (str): Input text that may contain markdown or plain text
        
    Returns:
        str: HTML formatted text with proper paragraph tags
    """
    # First, check if the text appears to contain markdown
    markdown_indicators = ['#', '*', '_', '`', '[', '>', '-', '+']
    has_markdown = any(indicator in text for indicator in markdown_indicators)
    
    if has_markdown:
        # Convert markdown to HTML using the markdown library
        md = markdown.Markdown(extensions=['extra'])
        html_content = md.convert(text)
        
        # Clean up the HTML using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Handle standalone text nodes by wrapping them in paragraphs
        for text_node in soup.find_all(text=True):
            if text_node.parent.name == 'body' or text_node.parent.name == '[document]':
                # Only wrap if it's not just whitespace
                if text_node.strip():
                    new_p = soup.new_tag('p')
                    text_node.wrap(new_p)
        
        return str(soup)
    else:
        # Handle plain text by splitting on newlines and wrapping in paragraph tags
        paragraphs = text.split('\n')
        return '\n'.join([f"<p>{p.strip()}</p>" for p in paragraphs if p.strip()])

def generate_html(book_dict):
    """Generate HTML content from book dictionary with markdown support.
    
    Args:
        book_dict (dict): Dictionary containing book content and structure
        
    Returns:
        str: Complete HTML document as string
    """
    # HTML template with styles
    html_template = """<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <style>
        @page {
            size: A4;
            margin: 2.5cm;
        }
        body {
            font-family: "Times New Roman", Times, serif;
            font-size: 12pt;
            line-height: 1.5;
            max-width: 21cm;
            margin: 0 auto;
            padding: 20px;
        }
        h1 {
            font-size: 24pt;
            text-align: center;
            margin-top: 40px;
            margin-bottom: 20px;
        }
        h2 {
            font-size: 18pt;
            margin-top: 30px;
        }
        .section-container {
            margin-bottom: 20px;
        }
        h3 {
            font-size: 14pt;
            margin-top: 20px;
            margin-bottom: 10px;
        }
        
        h4 {
            font-size: 13pt;
            margin-top: 20px;
            margin-bottom: 10px;
        }
        p {
            margin-bottom: 12pt;
            text-align: justify;
            orphans: 3;
            widows: 3;
            word-wrap: break-word;
            overflow-wrap: break-word;
            hyphens: auto;
            -webkit-hyphens: auto;
            -ms-hyphens: auto;
            page-break-inside: avoid;
        }
        .summary {
            margin: 20px 0;
            font-style: italic;
        }
        .chapter-summary {
            margin: 18px 0;
            font-style: italic;
            padding: 12px;
            background-color: #f8f8f8;
            border-left: 3px solid #333;
        }
        .page-break {
            page-break-after: always;
        }
        .chapter-container {
            margin-bottom: 40px;
        }
        .title-container {
            page-break-inside: avoid;
            margin-bottom: 30px;
        }
        hr {
            margin: 20px 0;
            border: none;
            border-top: 1px solid #000;
        }
        /* Additional styles for markdown elements */
        blockquote {
            margin: 15px 30px;
            padding-left: 15px;
            border-left: 3px solid #ccc;
        }
        code {
            font-family: monospace;
            background-color: #f5f5f5;
            padding: 2px 4px;
            border-radius: 3px;
        }
        pre {
            background-color: #f5f5f5;
            padding: 15px;
            border-radius: 5px;
            overflow-x: auto;
        }
        em {
            font-style: italic;
        }
        strong {
            font-weight: bold;
        }
    </style>
</head>
<body>
"""
    
    # Add title and main summary
    html = html_template
    html += "<div class='title-container'>\n"
    html += f"<h1>{book_dict['title']}</h1>\n"
    html += f"<div class='summary'>{text_to_paragraphs(book_dict['summary']['text'])}</div>\n"
    html += "</div>\n"
    html += "<hr>\n"

    html += "<div class='page-break'></div>"
    # Add chapter summaries overview
    for chapter_title, chapter in book_dict['content'].items():
        print(chapter_title)
        print(text_to_paragraphs(chapter['summary']['text']))
        print()
        print()

        html += "<div class='page-break'></div>"
        html += f"<div class='chapter-summary'><h3>{chapter_title}</h3>\n"
        html += f"{text_to_paragraphs(chapter['summary']['text'])}</div>\n"
    
    html += "<div class='page-break'></div>"

    # Process each chapter in detail
    for chapter_title, chapter in book_dict['content'].items():
        html += "<div class='chapter-container'>\n"
        html += f"<h2>{chapter_title}</h2>\n"
        html += "<hr>\n"
        
        # Process each section
        for section in chapter['content']:
            html += "<div class='section-container'>\n"
            html += f"<h3>{section['title']}</h3>\n"
            html += text_to_paragraphs(section['summary']['text']) + "\n"
            html += "</div>\n"

        html += "<div class='page-break'></div>"
            
        html += "</div>\n"
    
    html += "</body></html>"
    return html

def save_html(html_content, filename="book_summary.html"):
    """Save the generated HTML to a file
    
    Args:
        html_content (str): HTML content to save
        filename (str): Output filename
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)

In [144]:
html = generate_html(book_dict_sections)
save_html(html, filename="polinated_summary.html")

Introduction
<p>In the introductory chapter of "Leadership" by Henry Kissinger, leadership is explored as an indispensable force steering societies from their historical contexts toward desired futures. This involves careful decisions and strategic guidance to avert stagnation and inspire advancement. Leaders are challenged to harmonize past lessons with future aspirations, showcasing the importance of understanding societal values while fostering progress. They must educate and motivate without resorting to coercion, building capable teams to reflect and extend their strengths. Attributes like character and courage are crucial for navigating complex decisions and upholding values.</p>
<p>Leadership proves vital during transitional phases where societal norms might become obsolete. In such periods, leaders must discern which aspects of history to preserve, which to let go, and determine whether their society can endure sacrifices for future benefits. They operate in uncertain environme